In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from shutil import copyfile
import sys

In [2]:
import sys
sys.path.append("/home/arimitsu/anaconda3/envs/yolox/lib/python3.7/site-packages")

In [3]:
train = pd.read_csv('./train-10folds.csv')

In [4]:
train

,video_id,sequence,video_frame,sequence_frame,image_id,annotations,n_annotations,has_annotations,image_path,subsequence_id,fold
0,0,40258,0,0,0-0,[],0,False,../input/tensorflow-great-barrier-reef/train_i...,1,5
1,0,40258,1,1,0-1,[],0,False,../input/tensorflow-great-barrier-reef/train_i...,1,5
2,0,40258,2,2,0-2,[],0,False,../input/tensorflow-great-barrier-reef/train_i...,1,5
3,0,40258,3,3,0-3,[],0,False,../input/tensorflow-great-barrier-reef/train_i...,1,5
4,0,40258,4,4,0-4,[],0,False,../input/tensorflow-great-barrier-reef/train_i...,1,5
...,...,...,...,...,...,...,...,...,...,...,...
23496,2,29859,10755,2983,2-10755,[],0,False,../input/tensorflow-great-barrier-reef/train_i...,137,7
23497,2,29859,10756,2984,2-10756,[],0,False,../input/tensorflow-great-barrier-reef/train_i...,137,7
23498,2,29859,10757,2985,2-10757,[],0,False,../input/tensorflow-great-barrier-reef/train_i...,137,7
23499,2,29859,10758,2986,2-10758,[],0,False,../input/tensorflow-great-barrier-reef/train_i...,137,7


In [5]:
# train = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv')
train['pos'] = train.annotations != '[]'

In [25]:
!mkdir -p ./yolo_data/subseq_fold2/images/val
!mkdir -p ./yolo_data/subseq_fold2/images/train

!mkdir -p ./yolo_data/subseq_fold2/labels/val
!mkdir -p ./yolo_data/subseq_fold2/labels/train

In [26]:
fold = 0

annos = []
for i, x in train.iterrows():
    if x.fold == fold:
        mode = 'val'
    else:
        # train
        mode = 'train'
        if not x.pos: continue
        # val
    copyfile(f'../input/tensorflow-great-barrier-reef/train_images/video_{x.video_id}/{x.video_frame}.jpg',
                f'./yolo_data/subseq_fold{fold}/images/{mode}/{x.image_id}.jpg')
    if not x.pos:
        continue
    r = ''
    anno = eval(x.annotations)
    for an in anno:
#            annos.append(an)
        r += '0 {} {} {} {}\n'.format((an['x'] + an['width'] / 2) / 1280,
                                        (an['y'] + an['height'] / 2) / 720,
                                        an['width'] / 1280, an['height'] / 720)
    with open(f'./yolo_data/subseq_fold{fold}/labels/{mode}/{x.image_id}.txt', 'w') as fp:
        fp.write(r)

In [27]:
# test_df = train[train.fold == 9]
# annot_counts = 0
# for row in test_df.iterrows():
#     annot_counts += row[1]['n_annotations']
# print(annot_counts)

In [5]:
hyps = '''
# YOLOv5 by Ultralytics, GPL-3.0 license
# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.0025  # initial learning rate (SGD=1E-2, Adam=1E-3)
# optimizer: Adam #Optimizer
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4 
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.0  # image HSV-Hue augmentation (fraction)
hsv_s: 0.0  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.0  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.6  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.5  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 1.0  # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)
'''

In [6]:
data = '''
# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: ../yolo_data/subseq_fold0/  # dataset root dir
train: images/train  # train images (relative to 'path') 128 images
val: images/val  # val images (relative to 'path') 128 images
test:  # test images (optional)

# Classes
nc: 1  # number of classes
names: ['reef']  # class names


# Download script/URL (optional)
# download: https://ultralytics.com/assets/coco128.zip
'''

In [56]:
# %cd ..

/home/arimitsu/anaconda3/projects/cots_reef_comp_yolov5_high_res


In [105]:
# !git clone https://github.com/ultralytics/yolov5.git

In [7]:
with open('./yolov5/data/reef_f1_naive.yaml', 'w') as fp:
    fp.write(data)
with open('./yolov5/data/hyps/hyp.heavy.2.yaml', 'w') as fp:
    fp.write(hyps)

In [8]:
%cd yolov5

/home/arimitsu/anaconda3/projects/cots_reef_comp_yolov5_high_res/yolov5


In [9]:
!ls data/

Argoverse.yaml	      VOC.yaml	     hyps			  scripts
GlobalWheat2020.yaml  VisDrone.yaml  images			  xView.yaml
Objects365.yaml       coco.yaml      reef_f1_naive.yaml
SKU-110K.yaml	      coco128.yaml   reef_f1_naive_finetune.yaml


In [10]:
#Try out 3600 img size
!python train.py --img 1920 --batch 2 --epochs 10 --data reef_f1_naive.yaml --name l6_3600_uflip_vm5_f1 --weights yolov5m6.pt --hyp data/hyps/hyp.heavy.2.yaml --save-period 1

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5m6.pt, cfg=, data=reef_f1_naive.yaml, hyp=data/hyps/hyp.heavy.2.yaml, epochs=10, batch_size=2, imgsz=1920, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=l6_3600_uflip_vm5_f1, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 18 (delta 7), reused 11 (delta 7), pack-reused 4
Unpacking objects: 1

               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all       6707       3063      0.766      0.479      0.549      0.241

     Epoch   gpu_mem       box       obj       cls    labels  img_size
       4/9     7.78G   0.04027   0.08505         0        11      1920: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all       6707       3063      0.763      0.513       0.58      0.301

     Epoch   gpu_mem       box       obj       cls    labels  img_size
       5/9     7.78G   0.03723   0.08224         0        52      1920: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all       6707       3063      0.789      0.529      0.611      0.324

     Epoch   gpu_mem       box       obj       cls    labels  img_size
       6/9     7.78G   0.03632   0.08165         0        46      1920: 100%|███
               Class     Images     L

In [63]:
%cd ..

/home/arimitsu/anaconda3/projects/cots_reef_comp_yolov5_high_res


In [64]:
hyps = '''
# YOLOv5 by Ultralytics, GPL-3.0 license
# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.0025  # initial learning rate (SGD=1E-2, Adam=1E-3)
# optimizer: Adam #Optimizer
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4 
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.6  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.5  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 1.0  # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)
'''

In [65]:
data = '''
# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: ../yolo_data/subseq_fold0/  # dataset root dir
train: images/train  # train images (relative to 'path') 128 images
val: images/val  # val images (relative to 'path') 128 images
test:  # test images (optional)

# Classes
nc: 1  # number of classes
names: ['reef']  # class names


# Download script/URL (optional)
# download: https://ultralytics.com/assets/coco128.zip
'''

In [66]:
with open('./yolov5/data/reef_f1_naive.yaml', 'w') as fp:
    fp.write(data)
with open('./yolov5/data/hyps/hyp.heavy.2.yaml', 'w') as fp:
    fp.write(hyps)

In [67]:
%cd yolov5

/home/arimitsu/anaconda3/projects/cots_reef_comp_yolov5_high_res/yolov5


In [68]:
#Try out 3600 img size
!python train.py --img 1920 --batch 2 --epochs 10 --data reef_f1_naive.yaml --name l6_3600_uflip_vm5_f1 --weights yolov5m6.pt --hyp data/hyps/hyp.heavy.2.yaml --save-period 1

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5m6.pt, cfg=, data=reef_f1_naive.yaml, hyp=data/hyps/hyp.heavy.2.yaml, epochs=10, batch_size=2, imgsz=1920, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=l6_3600_uflip_vm5_f1, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 61 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.0-192-g436ffc4 torch 1.10.1 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)

hype

       5/9     7.78G    0.0243   0.01702         0         2      1920: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all       3250        623      0.779      0.647      0.685       0.33

     Epoch   gpu_mem       box       obj       cls    labels  img_size
       6/9     7.78G   0.02336   0.01652         0         5      1920: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all       3250        623      0.787       0.61      0.686      0.344

     Epoch   gpu_mem       box       obj       cls    labels  img_size
       7/9     7.78G   0.02265   0.01597         0         2      1920: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all       3250        623      0.818      0.636      0.695      0.343

     Epoch   gpu_mem       box       obj       cls    labels  img_size
       8/9     7.78G   0.02207   0.01

In [ ]:
# !python train.py --img 3600 --batch 1 --epochs 10 --data reef_f1_naive.yaml --name l6_3600_uflip_vm5_f1 --weights yolov5m6.pt --hyp data/hyps/hyp.heavy.2.yaml